In [1]:
import torch

X = torch.tensor([
[
    [1, 2],
    [3, 4]
],
[
    [1, 2],
    [3, 4]
]
])

X.shape

torch.Size([2, 2, 2])

In [2]:
X[-1].shape, X[-1]

(torch.Size([2, 2]),
 tensor([[1, 2],
         [3, 4]]))

In [3]:
context = X[-1].repeat(3, 1, 1)
context.shape

torch.Size([3, 2, 2])

In [4]:
for i in range(3):
    print(torch.cat([context, context], i).shape)

torch.Size([6, 2, 2])
torch.Size([3, 4, 2])
torch.Size([3, 2, 4])


In [5]:
# num_steps, batch_size, num_hiddens
# 在 2 号维度上拼接，扩展每个矩阵的列, 扩展了输入的样本的每个时间步上的值的向量长度
torch.cat([context, context], 2)

tensor([[[1, 2, 1, 2],
         [3, 4, 3, 4]],

        [[1, 2, 1, 2],
         [3, 4, 3, 4]],

        [[1, 2, 1, 2],
         [3, 4, 3, 4]]])

In [6]:
W = torch.tensor([
    [1, 2],
    [3, 4]
])

1, 2
1, 2

3, 4
3, 4

# batch_size, num_keys, key_size
K = torch.tensor([
    [[1, 2], [1, 2]],
    [[3, 4], [3, 4]],
])

In [10]:
torch.matmul(W, K)

tensor([[[ 3,  6],
         [ 7, 14]],

        [[ 9, 12],
         [21, 28]]])

In [11]:
# batch_size, num_queries, num_hiddens
# batch_size, num_keys_values, num_hiddens
# ==>
# batch_size, num_queries, 1, num_hiddens
# batch_size, 1, num_keys_values, num_hiddens

In [18]:
batch_size = 1
num_queries = 2
num_hiddens = 5
num_keys_values = 3

queries = torch.rand(size=(batch_size, num_queries, num_hiddens))
keys = torch.rand(size=(batch_size, num_keys_values, num_hiddens))

In [19]:
queries.shape, keys.shape

(torch.Size([1, 2, 5]), torch.Size([1, 3, 5]))

In [25]:
# 对于扩展成这样的维度后
# 广播的运行机制是
# --------------------
# 比如在 1 号维度，Y 的维度 < X 的维度
# 这时候就要让维度值小的 扩展成 维度值高的
# 所以在 1 号维度上 Y 向 X 看起 ==> 扩展方法是把更低维度的 2 号和 3 号上的内容复制多次堆叠在 1 号维度上
# --------------------
# 再比如 在 2 号维度上 X 的维度值 < Y 的维度值
# 因此扩展 X 的 2 号维度数值
# 扩展 2 号维度的数值，就要把比 2 号维度更低维度的所有维度的内容进行复制后堆叠在 2 号维度上
# 在这里比 2 号维度低的维度只有 3 号维度，因此把一个 num_hiddens 长的向量复制 num_keys_values 次堆叠在 2 号维度上

# 在三位世界中，第三个维度的长度为 1 ，那说明这个空间上的内容完全可以使用 2 维世界来承载
X = queries.unsqueeze(2)  # batch_size, num_queries,      1,             num_hiddens
Y = keys.unsqueeze(1)  # batch_size,         1,        num_keys_values,  num_hiddens
X.shape, Y.shape

(torch.Size([1, 2, 1, 5]), torch.Size([1, 1, 3, 5]))

In [21]:
res = X + Y  # 这样就实现了每一个查询在所有的 keys 上做加法运算

In [22]:
res.shape

torch.Size([1, 2, 3, 5])

In [24]:
res[0]

tensor([[[1.7006, 1.5571, 0.4660, 0.6582, 0.8745],
         [1.0644, 1.5856, 0.7767, 1.3422, 0.9221],
         [1.3725, 1.6271, 0.5387, 0.4978, 0.6640]],

        [[1.9832, 1.4424, 0.7791, 0.2784, 0.9185],
         [1.3470, 1.4709, 1.0898, 0.9624, 0.9661],
         [1.6551, 1.5123, 0.8518, 0.1180, 0.7079]]])